# 4.02: Importing Feathered Data Into Pandas for EDA, Cleaning
---

In [5]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import string
import re
import feather
import csv
import os
import collections
from datetime import datetime
from pickletools import pickle

---
### 1\. Dry Run

In [3]:
path = '../../../Large Datasets/capstone/feather/'
cces0616_feather = feather.read_dataframe(path+'cces_cumul_0616_rds.feather')

In [4]:
cces0616_feather.head()

,year,case_id,weight,weight_cumulative,state,st,cd,dist,dist_up,cong,...,voted_gov_chosen,voted_gov_fec,rep_current,rep_icpsr,sen1_current,sen1_icpsr,sen2_current,sen2_icpsr,gov_current,gov_fec
0,2006,439219,1.851676,1.667581,North Carolina,NC,NC-10,10,10,109,...,None,None,Patrick T. McHenry (R),20522.0,Elizabeth Dole (R),40303.0,Richard Burr (R),29548.0,Michael Easley (D),NC5998
1,2006,439224,0.968308,0.872039,Ohio,OH,OH-3,3,3,109,...,Ted Strickland (D),OH19691,Michael R. Turner (R),20342.0,Mike DeWine (R),15020.0,George V. Voinovich (R),49903.0,Bob Taft (R),None
2,2006,439228,1.593441,1.435020,New Jersey,NJ,NJ-1,1,1,109,...,None,None,Robert E. Andrews (D),29132.0,Robert Menendez (D),29373.0,Frank R. Lautenberg (D),14914.0,Jon Corzine (D),NJ6395
3,2006,439237,1.398529,1.259486,Illinois,IL,IL-9,9,9,109,...,Rod Blagojevich (D),IL7,Janice D. Schakowsky (D),29911.0,Richard Durbin (D),15021.0,Barack Obama (D),40502.0,Rod Blagojevich (D),IL7
4,2006,439238,0.902890,0.813124,New York,NY,NY-22,22,22,109,...,Eliot Spitzer (D),NY19490,Maurice D. Hinchey (D),29380.0,Charles E. Schumer (D),14858.0,Hillary Rodham Clinton (D),40105.0,George Pataki (R),None


In [6]:
dictionary = collections.defaultdict()

In [7]:
cces0616_dtypes = cces0616_feather.dtypes.to_dict(into=dictionary)

In [8]:
cces0616_dtypes

defaultdict(None,
            {'year': dtype('int32'),
             'case_id': dtype('int32'),
             'weight': dtype('float64'),
             'weight_cumulative': dtype('float64'),
             'state': dtype('O'),
             'st': dtype('O'),
             'cd': dtype('O'),
             'dist': dtype('int32'),
             'dist_up': dtype('int32'),
             'cong': dtype('int32'),
             'cong_up': dtype('int32'),
             'zipcode': dtype('O'),
             'county_fips': dtype('O'),
             'tookpost': dtype('float64'),
             'weight_vv': dtype('float64'),
             'weight_vv_post': dtype('float64'),
             'starttime': datetime64[ns, UTC],
             'pid3': dtype('float64'),
             'pid7': dtype('float64'),
             'pid3_leaner': dtype('float64'),
             'gender': dtype('int32'),
             'birthyr': dtype('int32'),
             'age': dtype('int32'),
             'race': dtype('int32'),
             'educ': dtype(

### 2\. Importing the Whole Kit and Kaboodle

In [11]:
files = os.listdir(path)

In [14]:
files.pop(0)

'.DS_Store'

In [15]:
print(files)

['1984_labeled_processed_race.feather', '1988_labeled_processed_race.feather', '1992_labeled_processed_race.feather', '1996_labeled_processed_race.feather', '2000_labeled_processed_race.feather', '2004_labeled_processed_race.feather', 'anes_timeseries_2016.feather', 'anes_timeseries_2016_voteval.feather', 'anes_timeseries_cdf.feather', 'annen061229.feather', 'annen2004_processed.feather', 'avgincome_orig.feather', 'cces_cumul_0616_rds.feather', 'cumulative_2006_2016_dta.feather', 'fips.icpsr.cbs.naes.feather', 'region_dummies.feather', 'region_dummies_84.feather', 'region_indic_annen2000.feather']


In [ ]:
# for file in files:
#     feather.read_dataframe(path+file)

In [17]:
anes_timeseries_2016_feather = feather.read_dataframe(path+'anes_timeseries_2016.feather')

In [18]:
anes_timeseries_cdf_feather = feather.read_dataframe(path+'anes_timeseries_cdf.feather')

In [19]:
labelprocrace_2004 = feather.read_dataframe(path+'2004_labeled_processed_race.feather')

In [20]:
labelprocrace_2004.head()

,state,pres04,sex,race,age9,partyid,income,relign8,age60,age65,geocode,sizeplac,brnagain,attend,year,region,y
0,2.0,1.0,2.0,1.0,2.0,NaN,NaN,NaN,1.0,2.0,3.0,5.0,NaN,NaN,2004.0,4.0,0.0
1,2.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,3.0,5.0,NaN,NaN,2004.0,4.0,1.0
2,2.0,1.0,2.0,2.0,3.0,NaN,NaN,NaN,2.0,3.0,3.0,5.0,NaN,NaN,2004.0,4.0,0.0
3,2.0,1.0,2.0,2.0,3.0,NaN,NaN,NaN,2.0,3.0,3.0,5.0,NaN,NaN,2004.0,4.0,0.0
4,2.0,1.0,2.0,1.0,4.0,NaN,NaN,NaN,2.0,4.0,3.0,5.0,NaN,NaN,2004.0,4.0,0.0


In [21]:
labelprocrace_2004.isnull().sum()

state           0
pres04        543
sex           337
race         1233
age9          505
partyid      3285
income       7343
relign8      7035
age60         505
age65         505
geocode         0
sizeplac      115
brnagain    45524
attend      47667
year            0
region          0
y            1787
dtype: int64

In [35]:
anes_timeseries_cdf_feather.shape

(55674, 952)

In [36]:
anes_timeseries_2016_voteval = feather.read_dataframe(path+'anes_timeseries_2016_voteval.feather')

In [22]:
anes_timeseries_2016_feather.columns

Index(['version', 'V160001', 'V160101', 'V160101f', 'V160101w', 'V160102',
       'V160102f', 'V160102w', 'V160201', 'V160201f',
       ...
       'V168123', 'V168124', 'V168125', 'V168126', 'V168127', 'V168128',
       'V168129', 'V168130', 'V168131', 'V168132'],
      dtype='object', length=1290)

In [23]:
anes_timeseries_cdf_feather.columns

Index(['Version', 'VCF0004', 'VCF0006', 'VCF0006a', 'VCF0009x', 'VCF0010x',
       'VCF0011x', 'VCF0009y', 'VCF0010y', 'VCF0011y',
       ...
       'VCF9132', 'VCF9133', 'VCF9134', 'VCF9149', 'VCF9150', 'VCF9151',
       'VCF9152', 'VCF9153', 'VCF9154', 'VCF9155'],
      dtype='object', length=952)

In [24]:
income_vote = labelprocrace_2004.dropna(axis=0, subset=['income','pres04'], how='any')

In [25]:
income_vote = income_vote[(income_vote['pres04'] == 1.) | (income_vote['pres04'] == 2.)][['income','pres04']]

In [26]:
income_vote['income'] = income_vote['income'].apply(lambda x: str(x))

In [27]:
income_vote_dummies = pd.get_dummies(income_vote['income'])

In [29]:
income_vote = pd.concat([income_vote,income_vote_dummies], join='inner', axis=1)

In [30]:
income_vote['pres04'] = income_vote['pres04'].astype('int64')

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [32]:
list(income_vote.columns)

['income', 'pres04', '1.0', '2.0', '3.0', '4.0', '5.0']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(income_vote[['1.0','2.0','3.0','4.0','5.0']],income_vote['pres04'],
                                                    test_size=0.2, random_state=42)
lr = LogisticRegression(random_state=42)


In [34]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
(lr.coef_, lr.intercept_)

(array([[-0.44081222, -0.04883874,  0.12228555,  0.11128208,  0.28668769]]),
 array([0.03060435]))

In [36]:
y_pred = lr.predict(X_test)

In [37]:
accuracy_score(y_test, y_pred)

0.5424797803665504